# Selenium 국회의원 데이터 스크래핑

## Selenium 및 웹 드라이버 설치

In [1]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver /usr/bin')

     |████████████████████████████████| 911kB 2.8MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.

In [2]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

## 라이브러리 import

In [5]:
import os
import shutil
import time
import socket
import pandas as pd

from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException

# 국회의원 기본정보 스크래핑

In [26]:
def get_politician_info(dir_name, wd):

  profile_info = wd.find_element_by_class_name('depAnalProfileTbl')

  cols = profile_info.find_elements_by_tag_name('th')
  columns = [col.text for col in cols]

  infos = profile_info.find_elements_by_tag_name('td')
  detail_info = [info.text for info in infos]

  info_dic = { col:info for col, info in zip(columns, detail_info) }

  return info_dic

# 국회의원 뉴스 인용문

In [44]:
def get_news_quote(dir_name, name, wd, news_max=5):
  page_no = 0
  news_count = 0
  news_total = wd.find_element_by_id('newsInQoutTotalCount').text[1:-2]
  news_file = open(dir_name + '/' + name + '뉴스 인용문.txt','w')

  while True:
    try:
      page_no += 1
      paging = wd.find_element_by_id('newsInQuotListPaging')
      pagination = paging.find_element_by_class_name('pagination')
      pagination.find_element_by_link_text(str(page_no)).click()
      time.sleep(1)
      print(f'뉴스 인용문 {page_no} 페이지')

      box_list = wd.find_element_by_id('newsInQuotList')
      quotes = box_list.find_elements_by_tag_name('b')
      for quote in quotes:
        news_file.write(quote.text + '\n')
        news_count += 1
        print(f' {news_count}/{news_total}', [quote.text])

        if news_count >= news_max:
          break
      if news_count >= news_max:
        break

      next_button = pagination.find_elements_by_tag_name('li')[-1]
      
      if next_button.get_attribute('class') == 'disabled':
        break

      if page_no % 5 == 0:
        next_button.find_element_by_tag_name('a').click()
        time.sleep(1)

    except:
      break

  news_file.close()

## 국회의원 회의록

In [77]:
def get_record_quote(dir_name, name, wd, record_max=5):
  page_no = 0
  record_count = 0
  record_total = wd.find_element_by_id('recordTotalCount').text
  record_file = open(dir_name + '/' + name + '회의록 인용문.txt','w')

  while True:
    try:
      page_no += 1
      paging = wd.find_element_by_id('recordTableListPaging')
      pagination = paging.find_element_by_class_name('pagination')
      pagination.find_element_by_link_text(str(page_no)).click()
      time.sleep(1)
      print(f'회의록 인용문 {page_no} 페이지')

      record_table = wd.find_element_by_id('recordTableList')
      record_list = record_table.find_elements_by_tag_name('a')
      for record in record_list:
        record.click()
        time.sleep(1)

        quote_list = wd.find_element_by_id('birefTableList')
        quotes = quote_list.find_elements_by_tag_name('td')
        for quote in quotes:
          record_file.write(quote.text + '\n')
          
        record_count += 1
        print(f' {record_count}/{record_total}', [quote.text])

        if record_count >= record_max:
          break
      if record_count >= record_max:
        break

      next_button = pagination.find_elements_by_tag_name('li')[-1]
      
      if next_button.get_attribute('class') == 'disabled':
        break

      if page_no % 5 == 0:
        next_button.find_element_by_tag_name('a').click()
        time.sleep(1)

    except:
      break

  record_file.close()


## `scraping()`: 스크래핑

In [80]:
def scraping(dir_name, politician_max=3):
  page_no = 0
  politician_count = 0
  politician_df = pd.DataFrame()

  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  wd.execute_script('window.open("about:black","_blacnk");')
  tabs = wd.window_handles

  while True:

    try:
      page_no += 1
      wd.switch_to.window(tabs[0])

      url = f'https://www.bigkinds.or.kr/v2/depthAnalysis/assembly.do?page={page_no}'
      wd.get(url)

      politician_total = wd.find_element_by_xpath('//*[@id="contents"]/div/div[3]/div/button[1]/span').text[1:-1]
      politician_items = wd.find_elements_by_class_name('assembly-item')

      if not politician_items:
        break
      for item in politician_items:
        wd.switch_to.window(tabs[0])

        politician_count += 1
        print('---------------------------------------------------')
        print(f'국회의원 {politician_count}/{politician_total} 명')
        name = item.find_element_by_class_name('assembly-item__profile__name').text
        name = name.split('\n')[0]
        print(name)

        detail_link = item.find_element_by_tag_name('a').get_attribute('href')

        wd.switch_to.window(tabs[1])
        wd.get(detail_link)

        print('[프로필 이지미 다운로드]')
        profile_image = wd.find_element_by_class_name('depAnalProfileImg')
        image_src = profile_image.find_element_by_tag_name('img').get_attribute('src')
        file_name = dir_name + '/' + name + '.jpg'
        print(file_name)
        urlretrieve(image_src, file_name)

        print('[기본 정보 스크래핑]')
        politician_dic = get_politician_info(dir_name, wd)
        print(' ',politician_dic)
        politician_df = politician_df.append(politician_dic, ignore_index=True)

        print('[뉴스 인용문 스크래핑]')
        wd.find_element_by_id('depthAnalTab2_1').find_element_by_tag_name('a').click()
        get_news_quote(dir_name, name, wd)

        print('[회의록 인용문 스크래핑]')
        get_record_quote(dir_name, name, wd)

        if politician_count >= politician_max:
          break

      if politician_count >= politician_max:
        break

    except AttributeError as e:
      print(e)
      break

    except NoSuchElementException as e:
      print(e)
      break

  wd.close()

  return politician_df


# 스크래핑 시작

In [81]:
dir_name = './politician'
if os.path.isdir(dir_name):
  shutil.rmtree(dir_name)

os.makedirs(dir_name)
print(f'{dir_name} 디렉토리 생성')

politician_df = scraping(dir_name)

./politician 디렉토리 생성
---------------------------------------------------
국회의원 1/300 명
강기윤 (姜起潤)
[프로필 이지미 다운로드]
./politician/강기윤 (姜起潤).jpg
[기본 정보 스크래핑]
  {'이름': '강기윤(姜起潤)', '이름(영문)': 'KANG GIYUN', '출생일': '1960-06-04', '정당': '미래통합당', '지역구': '경남 창원시성산구', '소속위원회': '보건복지위원회', '당선기록': '재선(19대, 21대)', '사무실전화': '02-784-1751', '홈페이지': 'http://blog.naver.com/ggotop', '이메일': 'ggotop@naver.com', '보좌관': '강종길 , 김홍광', '경력': '[학력] 마산공고(26회) 창원대학교 행정학과 중앙대학교 행정대학원 지방의회과 석사 창원대학교 대학원 행정학 박사 [경력] 보건복지위원회 위원 미래통합당 소상공인살리기 특별위원회 부위원장 미래통합당 경남도당 민생특위 위원장 제19대 국회의원 (새누리당/경남 창원시 성산구) 새누리당 원내부대표'}
[뉴스 인용문 스크래핑]
뉴스 인용문 1 페이지
 1/40 ['외국인 건강보험증을 별도로 만들어 본인 확인 절차를 강화하는 동시에 공단이 출입국 및 세무 당국 등과 협조해 의료기관과 실시간으로 수급자격을 확인·공유할 수 있는 시스템을 구축해야 한다']
 2/40 ['어제 3개 상임위에서 기습적으로 법안을 단독 처리한 것엔 야당 의원으로서 유감 이런 일이 우리 위원회에선 일어나면 안된다는 절박감이 있어 자리에 함께 하게 됐다. 원래 법안은 법안소위 심사를 거쳐야 하지만 코로나19 사태가 너무 긴박해 하루빨리 관련 법을 제정해야겠다는 생각을 했다 국민과 국가를 위하는 데에는 여야가 있을 수 없다 야당 간사지만 어떤 이..']
 3/40 ['전남도 해야 하는데, 더 열악한 부분이 우선돼야 한다. 그래야 합리적이고 객관적으로 결정했다고 듣지 않겠냐 

## 스크래핑 확인

In [82]:
politician_df

,경력,당선기록,보좌관,사무실전화,소속위원회,이름,이름(영문),이메일,정당,지역구,출생일,홈페이지
0,[학력] 마산공고(26회) 창원대학교 행정학과 중앙대학교 행정대학원 지방의회과 석사...,"재선(19대, 21대)","강종길 , 김홍광",02-784-1751,보건복지위원회,강기윤(姜起潤),KANG GIYUN,ggotop@naver.com,미래통합당,경남 창원시성산구,1960-06-04,http://blog.naver.com/ggotop
1,,초선(21대),"박홍규 , 정운태",,국방위원회,강대식(姜大植),KANG DAESIK,,미래통합당,대구 동구을,1959-11-02,
2,2018. 10. ~ 2019. 5. 민주연구원 자치발전연구센터 본부장 2016. ...,초선(21대),유진우,02-784-2747~9,교육위원회,강득구(姜得求),KANG DEUKGU,mainsail440@daum.net,더불어민주당,경기 안양시만안구,1963-05-27,https://blog.naver.com/dulipapa


In [83]:
!ls politician/

'강대식 (姜大植).jpg'		    '강득구 (姜得求)회의록 인용문.txt'
'강대식 (姜大植)뉴스 인용문.txt'    '강기윤 (姜起潤).jpg'
'강대식 (姜大植)회의록 인용문.txt'  '강기윤 (姜起潤)뉴스 인용문.txt'
'강득구 (姜得求).jpg'		    '강기윤 (姜起潤)회의록 인용문.txt'
'강득구 (姜得求)뉴스 인용문.txt'


In [84]:
from google.colab import files
files.download('./politician/강대식 (姜大植).jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [85]:
!cat './politician/강기윤 (姜起潤)뉴스 인용문.txt'

외국인 건강보험증을 별도로 만들어 본인 확인 절차를 강화하는 동시에 공단이 출입국 및 세무 당국 등과 협조해 의료기관과 실시간으로 수급자격을 확인·공유할 수 있는 시스템을 구축해야 한다
어제 3개 상임위에서 기습적으로 법안을 단독 처리한 것엔 야당 의원으로서 유감 이런 일이 우리 위원회에선 일어나면 안된다는 절박감이 있어 자리에 함께 하게 됐다. 원래 법안은 법안소위 심사를 거쳐야 하지만 코로나19 사태가 너무 긴박해 하루빨리 관련 법을 제정해야겠다는 생각을 했다 국민과 국가를 위하는 데에는 여야가 있을 수 없다 야당 간사지만 어떤 이..
전남도 해야 하는데, 더 열악한 부분이 우선돼야 한다. 그래야 합리적이고 객관적으로 결정했다고 듣지 않겠냐 전남에는 전남대, 조선대 의대가 두 곳 있는데 경남에는 하나 밖에 없다 인구 대비 의대 정원을 보면 경남은 0.0002명인데 전남은 0.0008명이다 (의대 신설은) 이런 점들이 고려 돼야 한다. 인구 104만인 창원에는 의대가 없다 그런 시각으로..
몸이 불편한 환자의 어려운 상황을 고려할 때에 비급여진료비 지불 적정성을 제대로 판단하기 어려울 수 있다 현재는 환자의 신청이 있어야 심평원이 진료비 적정 여부 확인 서비스를 시행하고 있는바, 환자의 신청이 없어도 심평원이 ‘연간 진료비 지불 내역 적정 여부’를 심사하여 그 결과를 1년 등의 기간 단위로 환자와 병원에 통보할 수 있도록 제도를 개선해야 ..
몸이 불편한 환자의 어려운 상황을 고려할 때에 비급여진료비 지불 적정성을 제대로 판단하기 어려울 수 있다 현재는 환자의 신청이 있어야 심평원이 진료비 적정 여부 확인 서비스를 시행하고 있는바, 환자의 신청이 없어도 심평원이 ‘연간 진료비 지불 내역 적정 여부’를 심사하여 그 결과를 1년 등의 기간 단위로 환자와 병원에 통보할 수 있도록 제도를 개선해야 ..


In [86]:
!cat './politician/강기윤 (姜起潤)회의록 인용문.txt'


감염자의, 확진자도 그렇지만 치료한, 사망률도 보면 지역과 차이가 많이 납니다, 지역 간에. 그래서 이 부분을 해소하기 위해서는 의료수가의 지역과 도시 간의 차등 적용도 필요할 것 같고요. 이번에 4000명을 늘리게 된다면 단순하게 지금 현재 기존에 있는 의대에다가 증원하는 것밖에 안 됩니다. 그건 무슨 의미냐면, 저는 개인적으로 생각할 때 이번에 4000명을 늘리면서 지역거점병원을 만들어야 되겠다는 생각입니다. 그것을 통해서 공공의료를 담당하게 하는 그런 역할을 겸임하면 앞으로 많은 역병들이 생길 텐데요, 그것을 방지할 수 있겠다, 지금 의대가 생기더라도굉장히 한 15년 정도 돼야 되지 않겠습니까? 굉장히 선행적인 투자가 필요한 부분이 있거든요.그런데 지금 단순하게 우리가 투자비도 많이 들고 빨리 의사 수를 늘리기 위해서 기존에 있는대학에다 증원한다는 것은 이와 같은 지역의 의료 불균형을 해소하기는 굉장히 어렵습니다. 이번에 문제된 것은 감염병을 지역에서 일차적으로 예방하지 못했던 것이 제일 크고요, 그다음에 의료 불균형이 노정이 되었습니다. 이 두 마리 토끼를 잡기 위해서는 그런 쪽에 기초해서 반드시 설계가 돼야 된다는 생각입니다. 예를 들겠습니다. 지역언론에 회자된 겁니다.저는 전남에 그런 대학병원을 하지 말자 이 이야기 절대 아닙니다. 거기도 해야 됩니다. 해야 되는데 그보다 더한, 더 열악한 부분이 있다면 그열악한 부분이 우선 돼야 된다는 데 저는 방점을두고 싶습니다. 그래야 합리적으로 결정했다, 객관적으로 결정했다 하고 듣지 않겠습니까? 지금 단순하게 비교하겠습니다. 저는 이게, 전남을 제가 뭐 어떻게 이렇게 하는 것 아닙니다.단순 비교하기 위해서 갖고 오는 건데요. 거기에는 전남대 대학병원도 있고 또 조선대도 있지 않습니까? 전남을 보면 인구로 보면 한 300만쯤 돼요. 제가 살고 있는 경남을, 죄송합니다. 경남을보면 경남에는 의대가 하나밖에 없어요. 인구비율을 보니까 인구는 무려 334만이거든요. 인구로보면, 인구 대 의대 정원을 보면 저희 경